In [1]:
#reference: https://www.pinecone.io/learn/faiss-tutorial/
import requests
from io import StringIO
import pandas as pd

In [3]:
!pip install faiss-cpu

Unable to create process using 'C:\Users\Zhao Yunjie\Documents\Anaconda\python.exe "C:\Users\Zhao Yunjie\Documents\Anaconda\Scripts\pip-script.py" install faiss-cpu'


In [2]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [3]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [4]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [5]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [6]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.4 MB/s 
     |████████████████████████████████| 5.5 MB 34.2 MB/s 
     |████████████████████████████████| 1.3 MB 44.5 MB/s 
     |████████████████████████████████| 182 kB 20.2 MB/s 
     |████████████████████████████████| 7.6 MB 36.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=71d8d37902237d978b0ce046ba844c45172b1f5b2eb216095dfeef50e54e82dc
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [7]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

(4802, 768)

In [8]:
sentence_embeddings

array([[-0.30220905,  0.23914625, -0.00655681, ...,  0.6008637 ,
         0.21191132,  0.07855172],
       [-0.01182628, -0.39084747,  1.2222238 , ...,  0.02022929,
        -0.3998495 ,  0.20652676],
       [ 0.36431047, -0.36761868, -0.18783367, ...,  0.5631123 ,
         0.3752037 , -0.24743702],
       ...,
       [ 0.43691474, -0.35765442,  0.27323747, ..., -0.04546407,
         0.6242372 ,  0.17364147],
       [ 0.32340202, -1.2503321 ,  0.6675966 , ..., -0.18992968,
        -0.6088423 , -0.1116884 ],
       [-0.6103973 , -0.09019724, -0.39196172, ...,  1.1877282 ,
         0.03778893, -0.81253296]], dtype=float32)

In [11]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 59 kB/s 


In [18]:
import faiss

In [19]:
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d) # euclidean distance
#https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

In [20]:
# we load our embeddings and query like so
index.add(sentence_embeddings)

In [21]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [22]:
D, I = index.search(xq, k)  # search
print(I)

[[  82 4019 4289  579]]


In [23]:
data['sentence_A'].iloc[[82,4019,4289,579]]

82                         Two dogs are playing by a tree
4019           A boy is playing a game with wooden blocks
4289    A man on a yellow sport bike is doing a wheeli...
579                  A boy is whacking a man with a sword
Name: sentence_A, dtype: object